# Decorators
- If you have gotten this far in your python career congrats nerd.
- Here is [PEP318](https://www.python.org/dev/peps/pep-0318), it's all about decorators in python. Go a head try to read it. I have tried several times and it sucks either because it is confusing as shit or I'm stupid. IDK you decide...
- Some of the code below comes from this [pretty good stackoverflow example](http://stackoverflow.com/questions/114214/class-method-differences-in-python-bound-unbound-and-static/114267#114267)
- Nice explaination of [decorators](https://realpython.com/blog/python/primer-on-python-decorators/).

In [1]:
# Import pandas and numpy for illustration purposes.
import pandas as pd
import numpy as np

class TestData(object):
    
    def selfDataFrame(self):
        return pd.DataFrame(np.random.rand(10), columns=['A'])
    @classmethod
    def classDataFrame(cls):
        return pd.DataFrame(np.random.rand(10), columns=['A'])
    @staticmethod
    def staticDataFrame():
        return pd.DataFrame(np.random.rand(10), columns=['A'])

# myTest = TestData()
# TestData.selfDataFrame(object)
# TestData.classDataFrame()
try:
    TestData.classDataFrame(object)
except TypeError as err:
    print("Awww dang... @classmethod can'")
# TestData.staticDataFrame()

Awww dang... @classmethod can'


In [2]:
class Test(object):
    @classmethod
    def method_one(cls):
        print("Called method_one")
    @staticmethod
    def method_two():
        print("Called method_two")

# a_test = Test()
# a_test.method_one()
# a_test.method_two()
Test.method_two()

Called method_two


---
## [Python decorators, the right way: the 4 audiences of programming languages](https://codewithoutrules.com/2017/08/10/python-decorators/)

In [18]:
from threading import Lock

def synchronized(function):
    """
    Given a method, return a new method that acquires a
    lock, calls the given method, and then releases the
    lock.
    """
    def wrapper(self, *args, **kwargs):
        """A synchronized wrapper."""
        with self._lock:
            return function(self, *args, **kwargs)
    return wrapper

In [19]:
class ExampleSynchronizedClass:
    def __init__(self):
        self._lock = Lock()
        self._items = []

    # Problematic usage:
    def add(self, item):
        """Add a new item."""
        self._items.append(item)
    add = synchronized(add)

esc = ExampleSynchronizedClass()

esc.add(1)

In [20]:
class ExampleSynchronizedClass:
    def __init__(self):
        self._lock = Lock()
        self._items = []

    # Nicer decorator usage:
    @synchronized
    def add(self, item):
        """Add a new item."""
        self._items.append(item)

esc = ExampleSynchronizedClass()

In [13]:
esc.add()

## Decorators using wrapt.

In [23]:
import wrapt
from threading import Lock

@wrapt.decorator
def synchronized(function, self, args, kwargs):
    """
    Given a method, return a new method that acquires a
    lock, calls the given method, and then releases the
    lock.
    """
    with self._lock:
        return function(*args, **kwargs)


In [24]:
class ExampleSynchronizedClass:
    def __init__(self):
        self._lock = Lock()
        self._items = []

    # Nicer decorator usage:
    @synchronized
    def add(self, item):
        """Add a new item."""
        self._items.append(item)

esc = ExampleSynchronizedClass()

esc.add(1)

---
## Decorators can be used to assign properties and attributes to an instance.

In [1]:
import wrapt

@wrapt.decorator
def basic(function, self, args, kwargs):
    'A basic decorator.'
    self.hello = 'hello'

class Decked(object):
    @basic
    def __init__(self):
        pass

In [2]:
d = Decked()
d.hello

'hello'

----
## [Python decorators with optional argument](https://codereview.stackexchange.com/a/78873/127038)

In [33]:
def optional_arg_decorator(fn):
    def wrapped_decorator(*args, **kwargs):
        if len(args) == 1 and len(kwargs) == 0 and callable(args[0]):
            return fn(args[0])

        else:
            def real_decorator(decoratee):
                return fn(decoratee, *args, **kwargs)

            return real_decorator

    return wrapped_decorator

---
## Meta Classes (classes are their instances) with that of Super Classes (classes are their sub-classes).

- SO thread on [meta/super classes in python](https://stackoverflow.com/questions/33727217/subscriptable-objects-in-class#33728603)

In [60]:
class MetaCls(type):
   def __getitem__(cls, index):
       print("Using meta __getitem__ on classes that have my type")

# metaclass is defined in header:
class Base(metaclass=MetaCls):
    pass


Base[0]

Using meta __getitem__ on classes that have my type


In [61]:
class MyMetaClass(type):
    def __getitem__(cls, x):
        return getattr(cls, x)

    def __new__(cls, name, parents, dct):
        dct["__getitem__"] = cls.__getitem__
        return super().__new__(cls, name, parents, dct)


class Dog(metaclass=MyMetaClass):
    x = 10

d = Dog()
print(d['x'])

10


---

In [117]:
class Base(object):
    x = 0

Base.__getitem__

AttributeError: type object 'Base' has no attribute '__getitem__'

In [115]:
class Base(object):
    x = 0
    def __getitem__(self, value):
        return self.x.__dict__[value]

In [116]:
Base['x']

TypeError: 'type' object is not subscriptable

---

In [130]:
from itertools import islice

class Sliceable(object):
    """Sliceable(iterable) is an object that wraps 'iterable' and
    generates items from 'iterable' when subscripted. For example:

        >>> from itertools import count, cycle
        >>> s = Sliceable(count())
        >>> list(s[3:10:2])
        [3, 5, 7, 9]
        >>> list(s[3:6])
        [13, 14, 15]
        >>> next(Sliceable(cycle(range(7)))[11])
        4
        >>> s['string']
        Traceback (most recent call last):
            ...
        KeyError: 'Key must be non-negative integer or slice, not string'

    """
    def __init__(self, iterable):
        self.iterable = iterable

    def __getitem__(self, key):
        if isinstance(key, int) and key >= 0:
            return islice(self.iterable, key, key + 1)
        elif isinstance(key, slice):
            return islice(self.iterable, key.start, key.stop, key.step)
        else:
            raise KeyError("Key must be non-negative integer or slice, not {}"
                           .format(key))

In [131]:
from itertools import count, cycle
s = Sliceable(count())
list(s[3:10:2])

[3, 5, 7, 9]

In [132]:
list(s[0])

[10]

In [136]:
from itertools import islice

class Sliceable(object):
    """Sliceable(iterable) is an object that wraps 'iterable' and
    generates items from 'iterable' when subscripted. For example:

        >>> from itertools import count, cycle
        >>> s = Sliceable(count())
        >>> list(s[3:10:2])
        [3, 5, 7, 9]
        >>> list(s[3:6])
        [13, 14, 15]
        >>> next(Sliceable(cycle(range(7)))[11])
        4
        >>> s['string']
        Traceback (most recent call last):
            ...
        KeyError: 'Key must be non-negative integer or slice, not string'

    """
    def __init__(self, iterable):
        self.iterable = iterable

    def __getitem__(self, key):
        if isinstance(key, int) and key >= 0:
            return islice(self.iterable, key, key + 1)
        elif isinstance(key, slice):
            return islice(self.iterable, key.start, key.stop, key.step)
        elif isinstance(key, str):
            if int(key) >= 0:
                return islice(self.iterable, int(key), int(key) + 1)
            
        else:
            raise KeyError("Key must be non-negative integer or slice, not {}"
                           .format(key))

In [141]:
from itertools import count, cycle
s = Sliceable(count())
list(s[3:10:2])

[3, 5, 7, 9]

In [146]:
list(s[0])

[14]

In [147]:
list(s['0'])

[15]

---

In [180]:
class Container(object):
    
    def __init__(self,*args,**kwargs):
        self.value = 0
    
    def __getitem__(self, key):
        return self.__dict__[key]
    
    def __setitem__(self, key, value):
        self.__dict__[key] = value

In [181]:
c = Container()

In [182]:
c['value'] = 1

In [184]:
c.value

1

---

In [202]:
import wrapt

@wrapt.decorator
def basic(function, self, args, kwargs):
    'A basic decorator.'
    self.hello = 'hello'

In [214]:
import wrapt

@wrapt.decorator
def dict_like(function, self, *args, **kwargs):
    
    def __getitem__(self, key):
        return self.__dict__[key]
    
    def __setitem__(self, key, value):
        self.__dict__[key] = value
    
    setattr(self, '__getitem__', __getitem__)

In [215]:
class Decked(object):
    @dict_like
    def __init__(self):
        self.hello = 0
        pass

In [216]:
d = Decked()

In [217]:
d['hello']

TypeError: 'Decked' object is not subscriptable

In [218]:
class F(object):
    def __init__(self, fn):
        self.__dict__['fn'] = fn

    def __call__(self, *args, **kwargs):
        return self.fn(*args, **kwargs)

    def __getitem__(self, name):
        return name

    def __getattr__(self, name):
        return name

In [221]:
def foo():
  print('hello')

foo = F(foo)

---

In [19]:
import time
from functools import wraps


def function_timer(some_function):
    """Return the time a function takes to execute."""
    @wraps(some_function)
    def wrapper():
        t1 = time.time()
        some_function()
        t2 = time.time()
        return '{}: {}'.format(some_function.__name__, str((t2 - t1)))
    return wrapper

In [20]:
@function_timer
def my_function():
    num_list = []
    for num in (range(0, 100000)):
        num_list.append(num)
    print("\nSum of all the numbers: " + str((sum(num_list))))


print(my_function())


Sum of all the numbers: 4999950000
my_function: 0.015599727630615234


In [12]:
my_function.__name__

'my_function'